<a href="https://colab.research.google.com/github/ahnselim/2025_Winter_InternShip/blob/master/Paper_QA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
# @title ## 1. PDF2TXT2CSV
# @markdown Change Paper ==> txt ==> CSV
import PyPDF2
import re
import pandas as pd

def pdf_to_text(pdf_path, skip_start_pages=0, skip_last_pages=0, header_lines=1, footer_lines=1):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        num_pages = len(pdf_reader.pages)

        print(f"Total pages in PDF: {num_pages}")

        # Adjust the range to process the correct pages
        start_page = skip_start_pages
        end_page = num_pages - skip_last_pages

        for page_num in range(start_page, end_page):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()

            if page_text:
                print(f"Page {page_num + 1}: {len(page_text)} characters extracted")
                lines = page_text.splitlines(True)[header_lines:-footer_lines]
                text += "".join(lines)
            else:
                print(f"Page {page_num + 1} is empty or could not be read")

        return text


# Define the input and output file paths
pdf_file_path = "/content/3362743.3362963.pdf" # @param {type : "string"}
output_file_path_txt = "/content/paper_txt.txt" # @param {type : "string"}

# Extract text from the PDF file
# Adjust skip_start_pages and skip_last_pages as needed
raw_text = pdf_to_text(pdf_file_path, skip_start_pages=0, skip_last_pages=0, header_lines=2, footer_lines=1)

# Save the extracted text to a text file
with open(output_file_path_txt, 'w', encoding='utf-8') as f:
    f.write(raw_text)

print(f"Text extraction complete. Total characters extracted: {len(raw_text)}")

output_dataset_path_csv  ="/content/paper_csv.csv" # @param{type:"string"}
data=re.sub(r'[\n\t\r]',' ',raw_text)
sentences=re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',data)
sentences=[sentence.strip() for sentence in sentences if sentence.strip()]
unique_sentences=list(dict.fromkeys(sentences))

df=pd.DataFrame(unique_sentences,columns=['Text'])
df.to_csv(output_dataset_path_csv,index=False)

Total pages in PDF: 6
Page 1: 6171 characters extracted
Page 2: 5578 characters extracted
Page 3: 4214 characters extracted
Page 4: 6023 characters extracted
Page 5: 2765 characters extracted
Page 6: 5628 characters extracted
Text extraction complete. Total characters extracted: 29507


In [ ]:
# @title ##2.Paper Embedding
# @markdown Put your Semantic Search model and dataset

from sentence_transformers import SentenceTransformer
import csv
import torch

model_name = 'paraphrase-multilingual-mpnet-base-v2' # @param{type:"string"}
encoded_model_path = '/content/semantic_search_model.pt'# @param{type:"string"}
dataset_path_csv = '/content/paper_csv.csv' # @param{type:"string"}

bi_encoder = SentenceTransformer(model_name)

passages = []
with open(dataset_path_csv) as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        passages.append(row[0])

corpus_embeddings = bi_encoder.encode(
    passages, batch_size=32, convert_to_tensor=True, show_progress_bar=True)

torch.save(corpus_embeddings, encoded_model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# @title ##3.Qustion Embedding
# @markdown Write your Question

import csv
from sentence_transformers import SentenceTransformer, util
import torch

bi_encoder = SentenceTransformer(model_name)
semantic_search_model = torch.load(encoded_model_path)

passages = []
with open(dataset_path_csv) as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        passages.append(row[0])
question="What hardware platforms were used for testing?" # @param{type:"string"}
question_embedding = bi_encoder.encode(
    question, convert_to_tensor=True)

<ipython-input-9-e2be4f0d416c>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  semantic_search_model = torch.load(encoded_model_path)


In [ ]:
# @title ##4.Semantic Search
# @markdown #Here is your answer📖 (3)
import json
hits = util.semantic_search(
    question_embedding, semantic_search_model, top_k=3)
hits = hits[0]

result = {
    "search_results": [
        {
            "score": hit['score'],
            "text": passages[hit['corpus_id']]
        }
        for hit in hits
    ]
}


print(json.dumps(result, indent=4, ensure_ascii=False))

{
    "search_results": [
        {
            "score": 0.5700193643569946,
            "text": "We test our implementation on two embedded devices which are representative of typical IoT devices you might see in use: Raspberry Pi Zero[ 5], Photon[ 9]."
        },
        {
            "score": 0.5163748860359192,
            "text": "Ideally, our hier- archical models would run on these native computing platforms."
        },
        {
            "score": 0.510373592376709,
            "text": "However they evaluate their systems on relatively high powered devices which aren’t represen- tative of typical edge devices which have fewer resources."
        }
    ]
}
